# User params

In [1]:
max_samples = 1_000

# Data cleaning, enrichment and encoding

In [ ]:
import os, sys
sys.path.append(os.path.abspath('../src'))
from src.config import BASE_PATH_DATA, PATH_PROJECT
os.chdir(PATH_PROJECT)

file_path_train_raw = os.path.join(BASE_PATH_DATA, 'processed', 'train.parquet')
file_path_test_raw = os.path.join(BASE_PATH_DATA, 'processed', 'test.parquet')

file_path_train_encoded = os.path.join(BASE_PATH_DATA, 'processed', f'df_train_encoded.parquet')
file_path_test_encoded = os.path.join(BASE_PATH_DATA, 'processed', f'df_test_encoded.parquet')

In [3]:
import pandas as pd
from src.utils.io.io import load_dataframe

# read json file into dataframe
df_train = load_dataframe(file_path_train_raw)
df_test = load_dataframe(file_path_test_raw)

# limit to wanted sample size, random_state for reproducibility
df_train = df_train.sample(n=max_samples, random_state=42)
df_test = df_test.sample(n=max_samples, random_state=42)

display(df_train.head(1))

2024-08-06 20:07:44,774 - src.config - DEBUG - calling load_dataframe
2024-08-06 20:07:44,777 - src.config - INFO - Current working directory: C:\Users\alexi\WORKSPACE\interview_abwaab\ReviewOutliers
2024-08-06 20:08:05,508 - src.config - DEBUG - calling load_dataframe
2024-08-06 20:08:05,510 - src.config - INFO - Current working directory: C:\Users\alexi\WORKSPACE\interview_abwaab\ReviewOutliers


,rating,title_review,text,images_review,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,...,description,price,images_metadata,videos,store,categories,details,bought_together,subtitle,author
458084,5.0,Saved the day! And my wallet!,I just spent a LOT of money ordering Christmas...,[],B07V45LJQ5,B07V45LJQ5,AG7JXXZEUKYYOJL7KH7LIFXHDR7Q,1669316848325,2,True,...,[],29.97,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [' 8544771 Dryer Heating Element, 2...",AMI PARTS,"[Appliances, Parts & Accessories, Dryer Parts ...","{""Manufacturer"": ""AMI PARTS"", ""Part Number"": ""...",None,None,None


In [4]:
from src.utils.preprocessing.preprocessing import preprocess_data

# encode the data
df_train_encoded = preprocess_data(df_train, training=True)
saved_name = os.path.join(BASE_PATH_DATA, 'processed', 'df_test_enriched.parquet')
df_test_encoded = preprocess_data(df_test, training=False, saved_name=saved_name)

# print some stats
print(f'length of df_train_encoded dataset: {df_train_encoded.shape}')
print(f'length of df_test_encoded dataset: {df_test_encoded.shape}')
display(df_train_encoded.head(1))

# Save the combined dataset (if needed)
df_train_encoded.to_parquet(file_path_train_encoded, index=False)
df_test_encoded.to_parquet(file_path_test_encoded, index=False)

c:\Users\alexi\AppData\Local\pypoetry\Cache\virtualenvs\reviewoutliers-SwZO3ms--py3.12\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-06 20:08:22,747 - datasets - INFO - PyTorch version 2.3.1 available.
2024-08-06 20:08:26,587 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2024-08-06 20:08:26,596 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: Alibaba-NLP/gte-base-en-v1.5
2024-08-06 20:08:26,614 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): huggingface.co:443
2024-08-06 20:08:27,037 - urllib3.connectionpool - DEBUG - https://huggingface.co:443 "HEAD /Alibaba-NLP/gte-base-en-v1.5/resolve/main/modules.json HTTP/11" 200 0
2024-08-06 20:08:27,345 - urllib3.connectionpo

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-08-06 20:16:47,648 - src.config - DEBUG - computing metadata embeddings


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-08-06 20:19:21,596 - src.config - DEBUG - calling encode_data
2024-08-06 20:19:21,616 - src.config - DEBUG - calling preprocess_data
2024-08-06 20:19:21,617 - src.config - DEBUG - calling clean_enrich
2024-08-06 20:19:21,647 - src.config - DEBUG - computing review embeddings


...

2024-08-06 20:32:02,190 - src.config - DEBUG - computing metadata embeddings


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2024-08-06 20:35:19,893 - src.config - DEBUG - calling encode_data


length of df_train_encoded dataset: (1000, 303)
length of df_test_encoded dataset: (1000, 303)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300,feature_301,feature_302
0,0.080945,0.291911,0.315864,0.250526,0.1965,0.171611,0.610187,-0.389288,-0.41343,-0.13935,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
